## Initial Setup and Imports

In [ ]:
# Imports
from astroquery.mast import MastMissions
from astropy.coordinates import SkyCoord
import astropy

# Create a MastMissions object, assign mission to 'jwst' for James Webb Space Telescope data
jwst_mission = MastMissions(mission='jwst')

## Running Search for L3 Products

In [ ]:
# Query for an object
# Grabbing data level 3 products on crab nebula
results = jwst_mission.query_object('CRABNEBULA',radius=1, productLevel=3)

In [ ]:
# Determine what products are available for the first results
products = jwst_mission.get_product_list(results[1])

In [ ]:
# Download the associated products
#jwst_mission.download_products(products, download_dir = "path/to/desired/output/location")

## Setup mast-aladin-lite for display

In [ ]:
# Setup mast-aladin-lite for viewing footprints
from mast_aladin_lite import MastAladin

mast_aladin = MastAladin(
    target = "05 34 31.78 +22 01 02.6",
    zoom = 2,
)

mast_aladin

In [ ]:
# Add the L3 results table from initial query for display
mast_aladin.add_table(results)

## Determine Associated L2s

In [ ]:
# Parse the downloaded association file from above download
import json
file_path = "path/to/desired/output/location/associationfile_asn.json"
with open(file_path, 'r') as file:
     data = json.load(file)
associated_L2s = [x["expname"] for x in data["products"][0]["members"]]

# Remove ends of strings from L2 file-naming convention to match fileSetName in jwst MastMissions results table
associated_L2s = [x.strip("_mirimage_cal.fits") for x in associated_L2s]

In [ ]:
# Query the same object with larger radius to capture all potential associated L2s
followup_results = jwst_mission.query_object('CRABNEBULA',radius=5)

# Filter to exclude L3 results
filtered = followup_results[followup_results["productLevel"] != "3"]

# Create a mask to grab the L2s of interest we found above
import numpy as np
mask = np.isin(filtered["fileSetName"], associated_L2s)

# Apply the mask to get the filtered table, make sure we have the same number of results
filtered_data = filtered[mask]
len(filtered_data), len(associated_L2s)

In [ ]:
# Lastly, add those into our aladin viewer! We can see the L2s comprising the initial L3 in the aladin viewer now
aladin.add_table(filtered_data)